In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from vllm import LLM, SamplingParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np

/home/aicombined/miniconda3/envs/poc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-17 20:01:02,775	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


- Instruction Formatting 전처리 함수

In [2]:
def preprocess_text(x):
    return f"###입력:{x}\n\n###출력:"

- merge_16bit로 저장된 모델 weight의 폴더를 이용하여 llm를 로드

In [3]:
llm = LLM(model="huggingface_merged_weights/SOLAR/", enable_lora=True)

INFO 06-17 20:01:03 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='huggingface_merged_weights/SOLAR/', speculative_config=None, tokenizer='huggingface_merged_weights/SOLAR/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=huggingface_merged_weights/SOLAR/)
INFO 06-17 20:01:03 utils.py:660] Found nccl from library /home/aicombined/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-17 20:01:04 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-17 20:01:04 sele

- 자연어 생성을 위한 그리드 서치 디코딩 파라미터 선언

In [4]:
sampling_params = SamplingParams(max_tokens=256, temperature=0.5, top_k=50)

- nsmc 데이터셋 불러오기

In [5]:
with open('./evaluation_dataset/01_nsmc/ratings_test.txt', 'r') as f:
    nsmc_data = f.readlines()
    
sentences = []
labels = []

for data in nsmc_data:
    data_split = data.split('\t')
    sentence = data_split[1]
    label = data_split[2].split('\n')[0]

    if label == '0':
        label = '부정'
    elif label == '1':
        label = '긍정'

    sentences.append(sentence)
    labels.append(label)
    
sentences = sentences[1:].copy()
labels = labels[1:].copy()

- held-out test set 중 10000개만 언더샘플링 하여 성능 산출

In [6]:
_, sentences_un, _, labels_un = train_test_split(sentences, labels, test_size=0.2, random_state=42, stratify=labels)

- 입력 데이터 전처리

In [7]:
system_prompt = """문장에 대해 긍정 또는 부정으로만 대답하세요\n\n"""
texts = []

for sentence, label in zip(sentences_un, labels_un):
    instruction = sentence
    texts.append(preprocess_text(system_prompt + instruction))

- inference

In [8]:
outputs = llm.generate(texts, sampling_params)

Processed prompts: 100%|██████████| 10000/10000 [03:36<00:00, 46.19it/s]


- 생성 결과 전처리

In [9]:
preds = []
for output in outputs:
    generated_text = output.outputs[0].text
    preds.append(generated_text.split('$&%')[0])

In [10]:
# prediction의 값이 긍정 또는 부정이 아닌경우 (이상한 대답을 한 경우)는 일단 제외, 추후 전혀 다른 대답을 한 경우도 성능에 포함
preds_processed = []
labels_processed = []
for y_pred, y_true in zip(preds, labels_un):
    if y_pred == '긍정' or y_pred == '부정':
        preds_processed.append(y_pred)
        labels_processed.append(y_true)

- zero-shot 분류 성능

In [11]:
print(classification_report(labels_processed, preds_processed))

              precision    recall  f1-score   support

          긍정       0.90      0.61      0.72      4438
          부정       0.71      0.93      0.80      4546

    accuracy                           0.77      8984
   macro avg       0.80      0.77      0.76      8984
weighted avg       0.80      0.77      0.76      8984

